In [6]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, split, explode, count, col
from pyspark.sql.types import List, StringType
from itertools import combinations


In [7]:
spark = SparkSession.builder.appName("adj").getOrCreate()
path = "../../simulated_postgres/filtered_DE.csv"

In [ ]:
# der postgres driver muss so verknüpft werden:
# spark-submit --jars pfad/postgresql-42.7.2.jar basics.ipynb
# das geht in notebooks nicht. als .py funktioniert es


# env = {
#     "user": "bigdata",
#     "password": "bigdata",
#     "driver": "org.postgresql.Driver",
# }

# DB_url = "jdbc:postgresql://localhost:5432/mydatabase"

# # Read DataFrame from PostgreSQL
# df = spark.read \
#     .format("jdbc") \
#     .option("url", DB_url) \
#     .option("dbtable", "filtered_de") \  # Specify the table name
#     .option("user", env["user"]) \
#     .option("password", env["password"]) \
#     .option("driver", env["driver"]) \
#     .load()


In [8]:
df = spark.read.option("header",'True').option('delimiter', ',').csv(path)


In [9]:

df.show()
# df.dropDuplicates(['video_id']).show()
print((df.count(), len(df.columns)))

+-----------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------------+-----------------+
|   video_id|               title|         publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|comment_count|comments_disabled|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------------+-----------------+
|jhMP8RSv4ws|Final-Auftritt vo...|2024-02-17T21:40:00Z|UCvbR8mrSZ1BXf2LN...|     Das Supertalent|        24|     24.18.02|Supertalent|Das S...|    514610|          336|            False|
|xeLMS48vrZU|Anna Ermakova - '...|2024-02-17T21:50:00Z|UCvbR8mrSZ1BXf2LN...|     Das Supertalent|        24|     24.18.02|Supertalent|Das S...|    487691|          332|            False|
|CxpUiuI9O4s|Der unfassbare Fa...|2024-02-18T11:00:08Z|UCKGMHVipE

In [10]:
df = df.withColumn("tags",split("tags", "\|") )

<>:1: SyntaxWarning: invalid escape sequence '\|'
<>:1: SyntaxWarning: invalid escape sequence '\|'
/var/folders/cn/t7fpnv554854x0vj7r6fxyjh0000gn/T/ipykernel_28633/2080672334.py:1: SyntaxWarning: invalid escape sequence '\|'
  df = df.withColumn("tags",split("tags", "\|") )


In [11]:
df.show()

+-----------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------------+-----------------+
|   video_id|               title|         publishedAt|           channelId|        channelTitle|categoryId|trending_date|                tags|view_count|comment_count|comments_disabled|
+-----------+--------------------+--------------------+--------------------+--------------------+----------+-------------+--------------------+----------+-------------+-----------------+
|jhMP8RSv4ws|Final-Auftritt vo...|2024-02-17T21:40:00Z|UCvbR8mrSZ1BXf2LN...|     Das Supertalent|        24|     24.18.02|[Supertalent, Das...|    514610|          336|            False|
|xeLMS48vrZU|Anna Ermakova - '...|2024-02-17T21:50:00Z|UCvbR8mrSZ1BXf2LN...|     Das Supertalent|        24|     24.18.02|[Supertalent, Das...|    487691|          332|            False|
|CxpUiuI9O4s|Der unfassbare Fa...|2024-02-18T11:00:08Z|UCKGMHVipE

In [12]:
videosAsListOfTags = df.select("tags").rdd.flatMap(lambda x: x).collect()
len(videosAsListOfTags)

1000

In [13]:
all_tags = df.select("video_id", explode("tags").alias("tags"))
all_tags.show()

+-----------+--------------------+
|   video_id|                tags|
+-----------+--------------------+
|jhMP8RSv4ws|         Supertalent|
|jhMP8RSv4ws|     Das Supertalent|
|jhMP8RSv4ws|         Deutschland|
|jhMP8RSv4ws|                 RTL|
|jhMP8RSv4ws|       Bruce Darnell|
|jhMP8RSv4ws|       Dieter Bohlen|
|jhMP8RSv4ws|        Super Talent|
|jhMP8RSv4ws|     Daniel Hartwich|
|jhMP8RSv4ws|                  TV|
|jhMP8RSv4ws|          Television|
|jhMP8RSv4ws|           Fernsehen|
|jhMP8RSv4ws|             Casting|
|jhMP8RSv4ws|      Modern Talking|
|jhMP8RSv4ws|Germany's Got Talent|
|jhMP8RSv4ws|          Got Talent|
|jhMP8RSv4ws|             Germany|
|jhMP8RSv4ws|              Talent|
|jhMP8RSv4ws|        Lola Weipert|
|jhMP8RSv4ws|    Ehrlich Brothers|
|jhMP8RSv4ws|  Riccardo Simonetti|
+-----------+--------------------+
only showing top 20 rows



In [14]:
counted_tags = all_tags.groupBy("tags").agg(count("tags").alias("count"))

In [15]:
counted_tags = counted_tags.orderBy("count", ascending=False).limit(500)
topxtags = dict(counted_tags.rdd.map(lambda row: (row["tags"], row["count"])).collect())
tagToID = {k: i for i, (k, _) in enumerate(topxtags.items())}


In [16]:
adj_mat = (df.select("tags")
           .rdd # convert to rdd
           .flatMap(lambda row: combinations(row["tags"], 2)) # finds all combinations
           .filter(lambda tags: all(tagToID.get(tag) is not None for tag in tags)) # remove all combinations not in top500
           .map(lambda tags: ((tagToID[tags[0]], tagToID[tags[1]]), 1)) #set count 1 and use touple
           .reduceByKey(lambda x, y: x + y) # all all cvalid combinatiosn together
           .toDF(["tags_pair", "count"])) #return to df
adj_mat.show()


+----------+-----+
| tags_pair|count|
+----------+-----+
|{176, 220}|   10|
|  {176, 5}|   10|
| {176, 14}|   10|
|{176, 185}|   10|
|{176, 218}|   10|
|{176, 233}|   10|
|{176, 182}|   10|
| {176, 73}|   10|
|{176, 225}|   10|
|{176, 131}|   10|
|{176, 136}|   10|
|{176, 179}|   10|
|{176, 212}|   10|
|{176, 237}|   10|
| {176, 60}|   10|
|{176, 210}|   10|
|{176, 177}|   10|
|{176, 175}|   10|
|{176, 216}|   10|
|{176, 236}|   10|
+----------+-----+
only showing top 20 rows



In [17]:
edge_table = adj_mat.withColumn("tag1", col("tags_pair").getItem("_1")) \
             .withColumn("tag2", col("tags_pair").getItem("_2")) \
             .drop("tags_pair")
edge_table.show()

+-----+----+----+
|count|tag1|tag2|
+-----+----+----+
|   10| 176| 220|
|   10| 176|   5|
|   10| 176|  14|
|   10| 176| 185|
|   10| 176| 218|
|   10| 176| 233|
|   10| 176| 182|
|   10| 176|  73|
|   10| 176| 225|
|   10| 176| 131|
|   10| 176| 136|
|   10| 176| 179|
|   10| 176| 212|
|   10| 176| 237|
|   10| 176|  60|
|   10| 176| 210|
|   10| 176| 177|
|   10| 176| 175|
|   10| 176| 216|
|   10| 176| 236|
+-----+----+----+
only showing top 20 rows



In [18]:
IDtoTag = {v: k for k, v in tagToID.items()}

def replace_id_with_name(id_value):
    return IDtoTag.get(id_value, id_value)

replace_id_with_name_udf = udf(replace_id_with_name, StringType())


new_edge = edge_table.withColumn("tag1", replace_id_with_name_udf(col("tag1")))
new_edge = new_edge.withColumn("tag2", replace_id_with_name_udf(col("tag2")))

new_edge = new_edge.withColumnRenamed('count', 'weight') \
      .withColumnRenamed('tag1', 'source') \
      .withColumnRenamed('tag2', 'destination')

new_edge = new_edge.orderBy("weight", ascending=False).limit(500)

new_edge.show()


+------+--------------------+--------------------+
|weight|              source|         destination|
+------+--------------------+--------------------+
|    32|            Fussball|                 DFB|
|    29|             Fußball|                 DFB|
|    25|                 RTL|      Lukas Podolski|
|    25|                 RTL|                RTL+|
|    25|                 RTL|               RTL +|
|    25|                 RTL|            RTL Plus|
|    25|                RTL+|               RTL +|
|    25|                RTL+|            RTL Plus|
|    24|              Soccer|                 DFB|
|    24|            Football|                 DFB|
|    22|            Fussball|         Deutschland|
|    22|             Fußball|          Bundesliga|
|    20|papaplatte highli...|papaplatte best o...|
|    20|papaplatte highli...| papaplatte realtalk|
|    20|papaplatte best o...| papaplatte realtalk|
|    20|            Fussball|              Soccer|
|    20|            Fussball|  

In [20]:
new_edge.toPandas().to_csv('../../simulated_postgres/edge_table.csv')

In [26]:
# Fiktiver Upload to PostgresSQL

# der postgres driver muss so verknüpft werden:
# spark-submit --jars pfad/postgresql-42.7.2.jar basics.ipynb
# das geht in notebooks nicht. als .py funktioniert es


env = {
    "user": "bigdata",
    "password": "bigdata",
    "driver": "org.postgresql.Driver",
}


DB_url = "jdbc:postgresql://localhost:5432/mydatabase"

# Write DataFrame to PostgreSQL
df.write \
    .format("jdbc") \
    .option("url", DB_url) \
    .option("dbtable", "edge_table") \
    .mode("overwrite") \
    .option("user", env["user"]) \
    .option("password", env["password"]) \
    .option("driver", env["driver"]) \
    .save()

Py4JJavaError: An error occurred while calling o229.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:588)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at or